# Step 5.: Machine Learning - Parameter Search / Optimization

In [1]:
# Import standard libraries
import os

# Import installed libraries
import numpy as np
import pandas as pd
import sklearn

from sklearn import preprocessing
from sklearn import svm
from sklearn.model_selection import GridSearchCV

In [2]:
# Let's mount the Google Drive, where we store files and models (if applicable, otherwise work
# locally)
try:
    from google.colab import drive
    drive.mount('/gdrive')
    core_path = "/gdrive/MyDrive/Colab/asteroid_taxonomy/"
except ModuleNotFoundError:
    core_path = ""

In [3]:
# Load the level 2 asteroid data
asteroids_df = pd.read_pickle(os.path.join(core_path, "data/lvl2/", "asteroids.pkl"))

In [4]:
# Now we add a binary classification schema, where we distinguish between e.g., X and non-X classes
asteroids_df.loc[:, "Class"] = asteroids_df["Main_Group"].apply(lambda x: 1 if x=="X" else 0)

In [5]:
# Allocate the spectra to one array and the classes to another one
asteroids_X = np.array([k["Reflectance_norm550nm"].tolist() for k in asteroids_df["SpectrumDF"]])
asteroids_y = np.array(asteroids_df["Class"].to_list())

In [6]:
# In this example we create a single test-training split with a ratio of 0.8 / 0.2
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)



result_df = pd.DataFrame([], columns=[])

for train_index, test_index in sss.split(asteroids_X, asteroids_y):
    
    X_train, X_test = asteroids_X[train_index], asteroids_X[test_index]
    y_train, y_test = asteroids_y[train_index], asteroids_y[test_index]

positive_class_weight = int(1.0 / (sum(y_train) / len(X_train)))
    
# Compute class weightning
positive_class_weight = int(1.0 / (sum(y_train) / len(X_train)))

param_grid = [
  {'C': [1], 'kernel': ['linear']},
  {'C': [1], 'kernel': ['rbf']},
 ]
svc = svm.SVC(class_weight={1: positive_class_weight})

# Import the preprocessing module

# Instantiate the StandardScaler (mean 0, standard deviation 1) and use the training data to fit
# the scaler
scaler = preprocessing.StandardScaler().fit(X_train)

# Transform now the training data
X_train_scaled = scaler.transform(X_train)

wclf = GridSearchCV(svc, param_grid, scoring='f1', verbose=3, cv=5)

# Perform the training
wclf.fit(X_train_scaled, y_train)

break

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END .............................C=1, kernel=linear; total time=   0.0s
[CV 2/5] END .............................C=1, kernel=linear; total time=   0.0s
[CV 3/5] END .............................C=1, kernel=linear; total time=   0.0s
[CV 4/5] END .............................C=1, kernel=linear; total time=   0.0s
[CV 5/5] END .............................C=1, kernel=linear; total time=   0.0s
[CV 1/5] END ................................C=1, kernel=rbf; total time=   0.0s
[CV 2/5] END ................................C=1, kernel=rbf; total time=   0.0s
[CV 3/5] END ................................C=1, kernel=rbf; total time=   0.0s
[CV 4/5] END ................................C=1, kernel=rbf; total time=   0.0s
[CV 5/5] END ................................C=1, kernel=rbf; total time=   0.0s


SyntaxError: 'break' outside loop (<ipython-input-6-d416bfc52868>, line 39)

In [7]:
wclf.best_score_

0.8455417253712865

In [15]:
final_clf = wclf.best_estimator_

In [16]:
# Scale the testing data ...
X_test_scaled = scaler.transform(X_test)

# ... and perform a predicition
y_test_pred = final_clf.predict(X_test_scaled)

In [12]:
# Import the confusion matrix and perform the computation
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, y_test_pred)

print(conf_mat)

# The order of the confusion matrix is:
#     - true negative (top left, tn)
#     - false positive (top right, fp)
#     - false negative (bottom left, fn)
#     - true positive (bottom right, tp)
tn, fp, fn, tp = conf_mat.ravel()

[[201  20]
 [  0  47]]


In [13]:
# Recall: ratio of correctly classified X Class spectra, considering the false negatives
# (recall = tp / (tp + fn))
recall_score = round(sklearn.metrics.recall_score(y_test, y_test_pred), 3)
print(f"Recall Score: {recall_score}")

# Precision: ratio of correctly classified X Class spectra, considering the false positives
# (precision = tp / (tp + fp))
precision_score = round(sklearn.metrics.precision_score(y_test, y_test_pred), 3)
print(f"Precision Score: {precision_score}")

# A combined score
f1_score = round(sklearn.metrics.f1_score(y_test, y_test_pred), 3)
print(f"F1 Score: {f1_score}")

Recall Score: 1.0
Precision Score: 0.701
F1 Score: 0.825


In [ ]:
# Finalize: Apply all data (take care of scaling!) on the training and rerun it. Save it afterwards for further computations
# Use https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html for partial fit